In [9]:
#######################Tensorflow 코드 시작부분
import librosa
import pyaudio #마이크를 사용하기 위한 라이브러리
import wave
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os

##### 변수 설정 부분 #####
DATA_PATH = "./data"
X_train = []#train_data 저장할 공간
X_test = []
Y_train = []
Y_test = []
tf_classes = 0

def load_wave_generator(path): 
       
    batch_waves = []
    labels = []
    X_data = []
    Y_label = []    
    global X_train, X_test, Y_train, Y_test, tf_classes
    
    folders = os.listdir(path)

    for folder in folders:
        if not os.path.isdir(path):continue #폴더가 아니면 continue                   
        files = os.listdir(path+"/"+folder)        
        print("Foldername :",folder,"-",len(files),"파일")
        #폴더 이름과 그 폴더에 속하는 파일 갯수 출력
        for wav in files:
            if not wav.endswith(".wav"):continue
            else:               
                #print("Filename :",wav)#.wav 파일이 아니면 continue
                y, sr = librosa.load(path+"/"+folder+"/"+wav)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T
                
                X_data.extend(mfcc)
               # print(len(mfcc))
                
                label = [0 for i in range(len(folders))]
                label[tf_classes] = 1
                
                for i in range(len(mfcc)):
                    Y_label.append(label)
                #print(Y_label)
        tf_classes = tf_classes+1
    #end loop
    print("X_data :",np.shape(X_data))
    print("Y_label :",np.shape(Y_label))
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_data), np.array(Y_label))

    xy = (X_train, X_test, Y_train, Y_test)
    np.save("./data.npy",xy)

load_wave_generator(DATA_PATH)

#t = np.array(X_train);
#print("!!!!!!!!",t,t.shape,X_train)
print(tf_classes,"개의 클래스!!")
print("X_train :",np.shape(X_train))
print("Y_train :",np.shape(Y_train))
print("X_test :",np.shape(X_test))
print("Y_test :",np.shape(Y_test))



Foldername : 0 - 20 파일
Foldername : 1 - 20 파일
Foldername : 2 - 20 파일
Foldername : 3 - 20 파일
Foldername : 4 - 20 파일
X_data : (54178, 20)
Y_label : (54178, 5)
5 개의 클래스!!
X_train : (40633, 20)
Y_train : (40633, 5)
X_test : (13545, 20)
Y_test : (13545, 5)


In [10]:
##################  화자인식 NN 버전 ##################
X_train, X_test, Y_train, Y_test = np.load("./data.npy")
X_train = X_train.astype("float")
X_test = X_test.astype("float")

# v1
tf.reset_default_graph() # 기존에 생성된 graph를 모두 삭제하고, reset시켜 중복되는 것을 막아준다. 
                         # context가 유지되는 주피터에서는 사용해야한다.
tf.set_random_seed(777)
learning_rate = 0.001
training_epochs = 200
keep_prob = tf.placeholder(tf.float32)
sd = 1 / np.sqrt(20) # standard deviation 표준편차(표본표준편차라 1/root(n))

#mfcc의 기본은 20
# 20ms일 때216은 각 mfcc feature의 열이 216
X = tf.placeholder(tf.float32, [None, 20])
# 
Y = tf.placeholder(tf.float32, [None, tf_classes])

# W = tf.Variable(tf.random_normal([216, 200]))
# b = tf.Variable(tf.random_normal([200]))

#1차 히든레이어
W1 = tf.get_variable("w1",
    #tf.random_normal([216, 180], mean=0, stddev=sd),
        shape=[20, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256], mean=0, stddev=sd), name="b1")
L1 = tf.nn.relu(tf.matmul(X, W1) + b1) # 1차 히든레이어는 'Relu' 함수를 쓴다.
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

# 2차 히든 레이어
W2 = tf.get_variable("w2",
    #tf.random_normal([180, 150], mean=0, stddev=sd),
         shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256], mean=0, stddev=sd), name="b2")
L2 = tf.nn.tanh(tf.matmul(L1, W2) + b2) # 2차 히든레이어는 'Relu' 함수를 쓴다.
L2 = tf.nn.dropout(L2, keep_prob = keep_prob)

# 3차 히든 레이어
W3 = tf.get_variable("w3",
    #tf.random_normal([150, 100], mean=0, stddev=sd),
            shape=[256, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([256], mean=0, stddev=sd), name="b3")
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3) # 3차 히든레이어는 'Relu' 함수를 쓴다.
L3 = tf.nn.dropout(L3, keep_prob = keep_prob)

# 4차 히든 레이어
W4 = tf.get_variable("w4",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[256, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([128], mean=0, stddev=sd), name="b4")
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4) # 4차 히든레이어는 'Relu' 함수를 쓴다.
L4 = tf.nn.dropout(L4, keep_prob = keep_prob)

# 5차 히든 레이어
W5 = tf.get_variable("w5",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([128], mean=0, stddev=sd), name="b5")
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5) # 5차 히든레이어는 'Relu' 함수를 쓴다.
L5 = tf.nn.dropout(L5, keep_prob = keep_prob)

# 6차 히든 레이어
W6 = tf.get_variable("w6",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([128], mean=0, stddev=sd), name="b6")
L6 = tf.nn.relu(tf.matmul(L5, W6) + b6) # 6차 히든레이어는 'Relu' 함수를 쓴다.
L6 = tf.nn.dropout(L6, keep_prob = keep_prob)

# 7차 히든 레이어
W7 = tf.get_variable("w7",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([128], mean=0, stddev=sd), name="b7")
L7 = tf.nn.relu(tf.matmul(L6, W7) + b7) # 7차 히든레이어는 'Relu' 함수를 쓴다.
L7 = tf.nn.dropout(L7, keep_prob = keep_prob)

# 최종 레이어
W8 = tf.get_variable("w8", 
    #tf.random_normal([50, tf_classes], mean=0, stddev=sd),
            shape=[128, tf_classes],
                     initializer=tf.contrib.layers.xavier_initializer())
b8 = tf.Variable(tf.random_normal([tf_classes], mean=0, stddev=sd), name="b8")
hypothesis = tf.matmul(L7, W8) + b8



#cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))


batch_size=1
x_len = len(X_train)
#짝수
if(x_len%2==0):
    batch_size = 2
elif(x_len%3==0):
    batch_size = 3
elif(x_len%4==0):
    batch_size = 4
else:
    batch_size = 1

split_X = np.split(X_train,batch_size)
split_Y = np.split(Y_train,batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
    
for epoch in range(training_epochs):
    avg_cost = 0
    for i in range(batch_size):
        batch_xs = split_X[i]
        batch_ys = split_Y[i]
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / batch_size
        #if(epoch%10==0):
    print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))

print('Learning Finished!')

Epoch: 0000 cost = 1.878391266
Epoch: 0001 cost = 1.734129310
Epoch: 0002 cost = 1.693499923
Epoch: 0003 cost = 1.675191641
Epoch: 0004 cost = 1.657896996
Epoch: 0005 cost = 1.646477103
Epoch: 0006 cost = 1.637713671
Epoch: 0007 cost = 1.635173202
Epoch: 0008 cost = 1.626531243
Epoch: 0009 cost = 1.627596855
Epoch: 0010 cost = 1.622805476
Epoch: 0011 cost = 1.620999455
Epoch: 0012 cost = 1.620400190
Epoch: 0013 cost = 1.618559718
Epoch: 0014 cost = 1.614760518
Epoch: 0015 cost = 1.613138080
Epoch: 0016 cost = 1.610566735
Epoch: 0017 cost = 1.609061956
Epoch: 0018 cost = 1.606838346
Epoch: 0019 cost = 1.600846648
Epoch: 0020 cost = 1.598018527
Epoch: 0021 cost = 1.594308496
Epoch: 0022 cost = 1.588436961
Epoch: 0023 cost = 1.579231143
Epoch: 0024 cost = 1.572331786
Epoch: 0025 cost = 1.561043382
Epoch: 0026 cost = 1.550191045
Epoch: 0027 cost = 1.540086150
Epoch: 0028 cost = 1.528746605
Epoch: 0029 cost = 1.519275784
Epoch: 0030 cost = 1.509343624
Epoch: 0031 cost = 1.502238750
Epoch: 0

In [11]:
#학습만 반복 코스트 보며 설정
for epoch in range(training_epochs):
    avg_cost = 0
    for i in range(batch_size):
        batch_xs = split_X[i]
        batch_ys = split_Y[i]
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / batch_size
        #if(epoch%10==0):
    print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))

print('Learning Finished!')


Epoch: 0000 cost = 0.407995820
Epoch: 0001 cost = 0.401891917
Epoch: 0002 cost = 0.399526179
Epoch: 0003 cost = 0.398920059
Epoch: 0004 cost = 0.395492345
Epoch: 0005 cost = 0.393142670
Epoch: 0006 cost = 0.394861847
Epoch: 0007 cost = 0.390815854
Epoch: 0008 cost = 0.388725936
Epoch: 0009 cost = 0.386350244
Epoch: 0010 cost = 0.386486977
Epoch: 0011 cost = 0.383720458
Epoch: 0012 cost = 0.379267454
Epoch: 0013 cost = 0.382202148
Epoch: 0014 cost = 0.375690430
Epoch: 0015 cost = 0.376492321
Epoch: 0016 cost = 0.382158220
Epoch: 0017 cost = 0.377438933
Epoch: 0018 cost = 0.375201941
Epoch: 0019 cost = 0.371289164
Epoch: 0020 cost = 0.368444264
Epoch: 0021 cost = 0.368222177
Epoch: 0022 cost = 0.366998792
Epoch: 0023 cost = 0.373307526
Epoch: 0024 cost = 0.367742032
Epoch: 0025 cost = 0.365212321
Epoch: 0026 cost = 0.362505466
Epoch: 0027 cost = 0.361817151
Epoch: 0028 cost = 0.362353921
Epoch: 0029 cost = 0.360128850
Epoch: 0030 cost = 0.355395585
Epoch: 0031 cost = 0.358767420
Epoch: 0

In [18]:
#학습만 반복 코스트 보며 설정 2
for epoch in range(training_epochs):
    avg_cost = 0
    for i in range(batch_size):
        batch_xs = split_X[i]
        batch_ys = split_Y[i]
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / batch_size
        #if(epoch%10==0):
    print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))

print('Learning Finished!')


Epoch: 0000 cost = 0.251638025
Epoch: 0001 cost = 0.254545093
Epoch: 0002 cost = 0.253473490
Epoch: 0003 cost = 0.254916757
Epoch: 0004 cost = 0.255413055
Epoch: 0005 cost = 0.249968037
Epoch: 0006 cost = 0.255346507
Epoch: 0007 cost = 0.252484292
Epoch: 0008 cost = 0.253185779
Epoch: 0009 cost = 0.251344293
Epoch: 0010 cost = 0.249671608
Epoch: 0011 cost = 0.253867328
Epoch: 0012 cost = 0.250390291
Epoch: 0013 cost = 0.251148999
Epoch: 0014 cost = 0.252305448
Epoch: 0015 cost = 0.253163755
Epoch: 0016 cost = 0.246926621
Epoch: 0017 cost = 0.251121789
Epoch: 0018 cost = 0.248001501
Epoch: 0019 cost = 0.251560181
Epoch: 0020 cost = 0.247300550
Epoch: 0021 cost = 0.248516560
Epoch: 0022 cost = 0.248166442
Epoch: 0023 cost = 0.248024672
Epoch: 0024 cost = 0.247982278
Epoch: 0025 cost = 0.243786588
Epoch: 0026 cost = 0.246818632
Epoch: 0027 cost = 0.246295586
Epoch: 0028 cost = 0.244948179
Epoch: 0029 cost = 0.244970649
Epoch: 0030 cost = 0.248966575
Epoch: 0031 cost = 0.244330108
Epoch: 0

In [19]:
saver = tf.train.Saver()
saver.save(sess, './my_voice_model2')

'./my_voice_model2'

In [24]:
y, sr = librosa.load("./test_정유경.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 정유경
1 배철수
2 이윤진
3 강정윤
4 임찬주
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[0] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))


#correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))
#print("Label :",sess.run(tf.argmax(Y_test,1)))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("predict")
print(pd.value_counts(pd.Series(sess.run(tf.argmax(hypothesis, 1),
                                    feed_dict={X: X_test, keep_prob:1}))))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))


(582, 20)
(582, 5)
predict
0    548
1     20
4     10
3      3
2      1
dtype: int64
Accuracy:  0.9415808
